# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

In [1]:
import xlwings as xw

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [ ]:
val=[sht.range('G7:M7').value,sht.range('G8:O8').value,sht.range('G9:O9').value,sht.range('G10:O10').value]
a=(sht.range('G14:O14').value)
for f in val:
    k=0
    ls=[]
    for i in f:
        if i==None:
            i=0
        ls.append(i*a[k])
        
        k=k+1
    print(sum(ls))

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [ ]:
sht.range('T7:T10').formula='=SUM(G7*G$14,H7*H$14,I7*I$14,J7*J$14,K7*K$14,L7*L$14,M7*M$14,N7*N$14,O7*O$14'

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

In [ ]:
sht.range('T7:T10').color=(255, 255, 255)
#(255, 255, 167)

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

In [3]:
import pandas as pd

In [ ]:
recipes=pd.read_csv('recipes_sample.csv')
reviews=pd.read_csv('reviews_sample.csv')
recipes=recipes[['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients']]

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [ ]:
df1 = data1.sample(n = int(len(recipes['name'])*0.05))
df1
df2 = data2.sample(n = int(len(reviews['user_id'])*0.05))
with pd.ExcelWriter('recipes.xlsx') as f:
    df1.to_excel(f,sheet_name='Рецепты')
    df2.to_excel(f,sheet_name='Отзывы')

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [ ]:
xf=xw.Book('recipes.xlsx')
rec=xw.sheets('Рецепты')
rec['H1'].value='seconds_assign'
rec['H2:H1501'].value=(df1['minutes'].values*60)[:,None]

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [ ]:
rec['I1'].value= 'seconds_formula'
rec['I2:I1501'].formula = '=D2*60'

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [ ]:
rec['H1:I1'].font.bold = True

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [ ]:
for i in rec['D2'].expand('down'):
    if i.value < 5: 
        i.color = (124, 252, 0)
    elif 5<=i.value<=10:
        i.color = (255,255,0)
    else: 
        i.color = (255,0,0)

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
rec['J1'].value = 'n_reviews'
fml = rec['J1'].formula = '=COUNTIF(Отзывы!D2:D6336, B2)'
rec.range('J2:J1501').formula = fml

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
rev = xf.sheets['Отзывы']
def validate():
    for i in range(2, df2.shape[0] + 1):
        if rev.range(f'C{i}').value not in df1['id']:
            rev.range(f'A{i}: F{i}').color = (220, 151, 255)
        elif rev.range(f'E{i}').value not in range(0,6):
            rev.range(f'A{i}:F{i}').color = (255, 0, 0)
validate()

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 